In [1]:
import numpy as np
import pandas as pd
import altair as alt

# Save a vega-lite spec and a PNG blob for each plot in the notebook
alt.renderers.enable("mimetype")
# Handle large data sets without embedding them in the notebook
alt.data_transformers.enable("data_server");

In [4]:
stinson2019_df = pd.read_pickle("data/stinson2019/stinson2019_df")
HF_weather = pd.read_csv('data/HF_weather/raw/hf001-08-hourly-m.csv', parse_dates=['datetime'])

/home/steffen/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (3,5,7,9,11,13,15,17,19,21,23,25,27,29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [29]:
stinson2019_df["jd"] = pd.DatetimeIndex(stinson2019_df["date"]).dayofyear
stinson2019_df["year_int"] = pd.DatetimeIndex(stinson2019_df["year"]).year
stinson2019_df["sugar_wt"] = stinson2019_df["sugar"]/100 * stinson2019_df["sap_wt"]
stinson2019_df["yearly_sap"] = np.nan
stinson2019_df["yearly_sugar"] = np.nan
stinson2019_df['tree_tap'] = stinson2019_df['tree']+ '_' + stinson2019_df['tap']
stinson2019_df = stinson2019_df.sort_values(by="date")

abs_measurement = 0

for year in stinson2019_df["year_int"].unique():
    year_df = stinson2019_df[stinson2019_df["year_int"] == year]

    for site in stinson2019_df[stinson2019_df["year_int"] == year]["site"].unique():
        for tree in stinson2019_df[
            (stinson2019_df["year_int"] == year) & (stinson2019_df["site"] == site)
        ]["tree"].unique():
            for tap in stinson2019_df[
                (stinson2019_df["year_int"] == year)
                & (stinson2019_df["site"] == site)
                & (stinson2019_df["tree"] == tree)
            ]["tap"].unique():
                tap_yr_index = stinson2019_df.index[
                    (stinson2019_df["year_int"] == year)
                    & (stinson2019_df["site"] == site)
                    & (stinson2019_df["tree"] == tree)
                    & (stinson2019_df["tap"] == tap)
                ].tolist()
                yearly_sap = 0
                yearly_sugar = 0
                for measurement in tap_yr_index:
                    yearly_sap = np.nansum(
                        [yearly_sap, stinson2019_df.loc[measurement, "sap_wt"]]
                    )
                    yearly_sugar = np.nansum(
                        [
                            yearly_sugar,
                            stinson2019_df.loc[measurement, "sap_wt"]
                            * stinson2019_df.loc[measurement, "sugar"]/100,
                        ]
                    )
                    stinson2019_df.loc[measurement, "yearly_sap"] = yearly_sap
                    stinson2019_df.loc[measurement, "yearly_sugar"] = yearly_sugar


#     yearly_sap = 0
#     yearly_sugar = 0
#     for measurement in list(range(len(year_df))):
#         yearly_sap+=stinson2019_df.iloc[abs_measurement,6]
#         yearly_sugar+=stinson2019_df.iloc[abs_measurement,6]*stinson2019_df.iloc[abs_measurement,5]
#         stinson2019_df.iloc[abs_measurement,13]=yearly_sap
#         stinson2019_df.iloc[abs_measurement,14]=yearly_sugar
#         abs_measurement+=1

In [16]:
stinson2019_df

,site,date,tree,species,tap,sugar,sap_wt,year,site_id,tree_id,tree_record_id,tap_record_id,jd,year_int,yearly_sap,yearly_sugar,tree_tap
0,Harvard Forest,2012-02-16,HF1,ACSA,A,2.7,4.24,2012-01-01,HF,HF.ACSA.HF1,HF.ACSA.HF1.2012-02-16,HF.ACSA.HF1.A.2012-02-16,47,2012,4.24,11.448,HF1_A
22,Harvard Forest,2012-02-16,HF43,ACSA,B,3.1,4.16,2012-01-01,HF,HF.ACSA.HF43,HF.ACSA.HF43.2012-02-16,HF.ACSA.HF43.B.2012-02-16,47,2012,4.16,12.896,HF43_B
21,Harvard Forest,2012-02-16,HF43,ACSA,A,3.2,8.42,2012-01-01,HF,HF.ACSA.HF43,HF.ACSA.HF43.2012-02-16,HF.ACSA.HF43.A.2012-02-16,47,2012,8.42,26.944,HF43_A
20,Harvard Forest,2012-02-16,HF41,ACSA,B,3.8,1.62,2012-01-01,HF,HF.ACSA.HF41,HF.ACSA.HF41.2012-02-16,HF.ACSA.HF41.B.2012-02-16,47,2012,1.62,6.156,HF41_B
19,Harvard Forest,2012-02-16,HF41,ACSA,A,3.7,2.36,2012-01-01,HF,HF.ACSA.HF41,HF.ACSA.HF41.2012-02-16,HF.ACSA.HF41.A.2012-02-16,47,2012,2.36,8.732,HF41_A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9092,Northern range - Quebec,2017-05-04,QC3,ACSA,A,NaN,0.50,2017-01-01,QC,QC.ACSA.QC3,QC.ACSA.QC3.2017-05-04,QC.ACSA.QC3.A.2017-05-04,124,2017,34.11,86.284,QC3_A
9091,Northern range - Quebec,2017-05-04,QC2,ACSA,A,1.6,0.11,2017-01-01,QC,QC.ACSA.QC2,QC.ACSA.QC2.2017-05-04,QC.ACSA.QC2.A.2017-05-04,124,2017,16.99,31.621,QC2_A
9090,Northern range - Quebec,2017-05-04,QC1,ACSA,A,1.7,1.18,2017-01-01,QC,QC.ACSA.QC1,QC.ACSA.QC1.2017-05-04,QC.ACSA.QC1.A.2017-05-04,124,2017,18.18,43.791,QC1_A
9105,Northern range - Quebec,2017-05-04,QC16,ACSA,A,1.6,0.34,2017-01-01,QC,QC.ACSA.QC16,QC.ACSA.QC16.2017-05-04,QC.ACSA.QC16.A.2017-05-04,124,2017,15.93,33.106,QC16_A


In [17]:
harvard_sap = stinson2019_df[stinson2019_df['site_id']=='HF']
harvard_sap['jd'].max()

102

In [27]:
weeks = list(range(0,106,7))
harvard_sap.groupby([harvard_sap['year'], pd.cut(harvard_sap['jd'], weeks)]).sum()

sugar  sap_wt       jd  year_int  yearly_sap  \
year       jd                                                         
2012-01-01 (0, 7]        NaN     NaN      NaN       NaN         NaN   
           (7, 14]       NaN     NaN      NaN       NaN         NaN   
           (14, 21]      NaN     NaN      NaN       NaN         NaN   
           (21, 28]      NaN     NaN      NaN       NaN         NaN   
           (28, 35]      NaN     NaN      NaN       NaN         NaN   
           (35, 42]      NaN     NaN      NaN       NaN         NaN   
           (42, 49]   151.50  230.19   3312.0  138828.0      447.25   
           (49, 56]   179.00  303.25   4807.0  185104.0     1643.81   
           (56, 63]    94.60  174.15   2691.0   92552.0     1309.08   
           (63, 70]   133.90  343.39   4692.0  138828.0     2744.59   
           (70, 77]   181.80  527.00   6670.0  185104.0     5685.10   
           (77, 84]    42.20   58.30   1817.0   46276.0     1636.28   
           (84, 91]      NaN     NaN      NaN       NaN         NaN   
           (91, 98]      NaN     NaN      NaN       NaN         NaN   
           (98, 105]     NaN     NaN      NaN       NaN         NaN   
2013-01-01 (0, 7]        NaN     NaN      NaN       NaN         NaN   
           (7, 14]       NaN     NaN      NaN       NaN         NaN   
           (14, 21]      NaN     NaN      NaN       NaN         NaN   
           (21, 28]      NaN     NaN      NaN       NaN         NaN   
           (28, 35]      NaN     NaN      NaN       NaN         NaN   
           (35, 42]      NaN     NaN      NaN       NaN         NaN   
           (42, 49]      NaN     NaN      NaN       NaN         NaN   
           (49, 56]      NaN     NaN      NaN       NaN         NaN   
           (56, 63]   588.60  662.54  12888.0  434808.0     2313.34   
           (63, 70]   578.75  898.32  14508.0  434808.0     6556.62   
           (70, 77]   229.60  232.86   7920.0  217404.0     5209.12   
           (77, 84]   376.30  571.16  11880.0  289872.0     8265.66   
           (84, 91]   431.10  915.35  18936.0  434808.0    18253.17   
           (91, 98]   328.80  633.62  17136.0  362340.0    18139.59   
           (98, 105]   35.60   50.02   3564.0   72468.0     3963.87   
2014-01-01 (0, 7]        NaN     NaN      NaN       NaN         NaN   
           (7, 14]       NaN     NaN      NaN       NaN         NaN   
           (14, 21]      NaN     NaN      NaN       NaN         NaN   
           (21, 28]      NaN     NaN      NaN       NaN         NaN   
           (28, 35]      NaN     NaN      NaN       NaN         NaN   
           (35, 42]      NaN     NaN      NaN       NaN         NaN   
           (42, 49]      NaN     NaN      NaN       NaN         NaN   
           (49, 56]   242.50  190.04   5724.0  217512.0      318.04   
           (56, 63]      NaN     NaN      NaN       NaN         NaN   
           (63, 70]   263.10  278.90   7380.0  217512.0     1036.80   
           (70, 77]    33.20   36.40   2556.0   72504.0      505.34   
           (77, 84]   331.30  252.37   8568.0  217512.0     2042.57   
           (84, 91]   479.70  596.56  15912.0  362520.0     5383.09   
           (91, 98]   388.20  486.16  13608.0  290016.0     6549.56   
           (98, 105]  166.90  258.70   7236.0  145008.0     4113.72   
2015-01-01 (0, 7]        NaN     NaN      NaN       NaN         NaN   
           (7, 14]       NaN     NaN      NaN       NaN         NaN   
           (14, 21]      NaN     NaN      NaN       NaN         NaN   
           (21, 28]      NaN     NaN      NaN       NaN         NaN   
           (28, 35]      NaN     NaN      NaN       NaN         NaN   

                      yearly_sugar  
year       jd                       
2012-01-01 (0, 7]              NaN  
           (7, 14]             NaN  
           (14, 21]            NaN  
           (21, 28]            NaN  
           (28, 35]            NaN  
           (35, 42]            NaN  
           (42, 49]       1112.736

In [14]:
pd.periodindex

TypeError: TimedeltaIndex() must be called with a collection of some kind, 1 was passed